# Формула расчёта смещения центров масс для ящика

### Исходные данные

In [27]:
# Характеристика 4-х осной ж/д платформы
L = 13400 # Длина пола
2870 # Ширина пола
Qv = 21 # Масса тары
wugr = 1310 # Высота пола от УГР
height_ct_vagon = 800 # Высота центра тяжести(ЦТ) от УГР
9720 # База платформы

delta = [150, 150, 150, 150] # расстояние между грузами (нет в записке)

# Вес - в тоннах.
# Длина, ширина, высота - в мм.
cargo = [ # грузы
    {'length': 1080, 'width': 1580, 'height': 390, 'weight': 395 / 1000},  
    {'length': 3650, 'width': 3320, 'height': 1500, 'weight': 6670 / 1000},
    {'length': 4100, 'width': 1720, 'height': 1150, 'weight': 1865 / 1000},
    {'length': 3870, 'width': 2890, 'height': 1020, 'weight': 4085 / 1000},
]
weightSum = sum(map(lambda x: x['weight'], cargo))

### Формулы

In [28]:
for index, item in enumerate(cargo):
    if index:
        prev = cargo[index - 1]
        item['ct'] = (item['length'] + prev['length']) / 2 + prev['ct'] + delta[index]
    else:
        item['ct'] = item['length'] / 2 + delta[index]


# Продольное смещение грузов в вагоне - в мм.
Lc = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) / weightSum)

# Продольное смещение грузов с вагоном - в мм.
Lcv = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) + Qv * L/2) / (weightSum + Qv)

# Общая высота ЦТ - в мм.
H = sum(map(lambda x: x['weight'] * (x['height']/2 + wugr), cargo)) / weightSum

print(Lc, Lcv, H)

433.3941605839418 165.8275760693814 1942.750672301191


In [15]:
for index, item in enumerate(cargo):
    if index:
        prev = cargo[index - 1]
        item['ct'] = (item['length'] + prev['length']) / 2 + prev['ct'] + delta[index]
    else:
        item['ct'] = item['length'] / 2 + delta[index]


# Продольное смещение грузов в вагоне - в мм.
Lc = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) / weightSum)

# Продольное смещение грузов с вагоном - в мм.
Lcv = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) + Qv * L/2) / (weightSum + Qv)

# Общая высота ЦТ - в мм.
H = sum(map(lambda x: x['weight'] * (x['height']/2 + wugr), cargo)) / weightSum

print(Lc, Lcv, H)

694.4967345370733 265.7320299867706 1942.750672301191


### Доп формулы

In [8]:
# Общая высота ЦТ
height_center_gravity_in_car = (sum(map(lambda x: x['weight'] * (x['height']/2 + wugr), cargo)) + Qv * height_ct_vagon)/ (weightSum + Qv)
height_center_gravity_in_car

1237.2453329413495

In [10]:
# Расчет наветренной поверхности.
# Значение 7 - константа
sum([item['length'] / 1000 * item['height'] / 1000 for item in cargo]) + 7

21.5586

### Генетический алгоритм

In [29]:
""" Модуль с реализацией генетического оператора скрещивания. """


import random


class Crossing:
    """ Cкрещивание. """

    def __init__(self, p: float, strategy: str):
        # Вероятность скрещивания.
        self.p = p
        self.strategy = strategy  # s | u

    def crossing(self, population: list[list]) -> list[list]:
        """ Скрещивание осуществляется с определённой вероятностью. """

        count_population = len(population)

        crossing_population = []
        for _ in range(count_population // 2):
            tmp_count_population = len(population) - 1

            # Случайный выбор 2 генотипов.
            idx_rand1 = random.randint(0, tmp_count_population)
            genotip1 = population.pop(idx_rand1)

            idx_rand2 = random.randint(0, tmp_count_population-1)
            genotip2 = population.pop(idx_rand2)

            # Скрещивание происходит в вероятноью p.
            if random.random() < self.p:

                if self.strategy == "s":
                    two_childs = self._singlePointCrossing(genotip1, genotip2)

                else:
                    two_childs =  self._uniformCrossing(genotip1, genotip2)

            else:
                two_childs = [genotip1, genotip2]

            # Добавление потомков в новое поколение.
            for child in two_childs:
                crossing_population.append(child)

        return crossing_population

    def _singlePointCrossing(self, genotip1: list, genotip2: list) -> list[list, list]:
        """ Одноточечное скрещивание. """

        count_variables = len(genotip1)

        two_childs = [[] for _ in range(2)]
        for i in range(count_variables):
            lenght = len(genotip1[i])
            point = random.randint(1, lenght-1)

            one_param_child1 = genotip1[i][:point] + genotip2[i][point:]
            one_param_child2 = genotip2[i][:point] + genotip1[i][point:]

            two_childs[0].append(one_param_child1)
            two_childs[1].append(one_param_child2)

        return two_childs

    def _uniformCrossing(self, genotip1: list, genotip2: list) -> list[list, list]:
        """ Равномерное скрещивание. """

        count_variables = len(genotip1)

        two_childs = [[] for _ in range(2)]
        for i in range(count_variables):
            lenght = len(genotip1[i])

            one_param_child1 = [random.choice([genotip1[i][k], genotip2[i][k]]) for k in range(lenght)]
            one_param_child2 = [random.choice([genotip1[i][k], genotip2[i][k]]) for k in range(lenght)]

            two_childs[0].append(one_param_child1)
            two_childs[1].append(one_param_child2)

        return two_childs


if __name__ == "__main__":

    crossing = Crossing(0.9, "u")

    population = [
        [[150], [150], [150], [150]],
        [[1, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 0], [1, 1, 1, 0, 1, 1]],
        [[0, 1, 1, 0, 1, 1], [0, 0, 1, 1, 0, 0], [1, 1, 1, 0, 1, 1]],
        [[1, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 1], [1, 1, 1, 0, 1, 1]],
        [[1, 1, 1, 1, 0, 0], [1, 0, 0, 0, 0, 1], [1, 1, 1, 0, 1, 1]],
        [[1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 1, 0], [1, 1, 1, 0, 1, 1]],
    ]
    # print(crossing.crossing(population))
    for i in crossing.crossing(population):
        print(i)

TypeError: object of type 'int' has no len()

In [ ]:
def run(delta):
    
    # Вес - в тоннах.
    # Длина, ширина, высота - в мм.
    cargo = [ # грузы
        {'length': 1080, 'width': 1580, 'height': 390, 'weight': 395 / 1000},  
        {'length': 3650, 'width': 3320, 'height': 1500, 'weight': 6670 / 1000},
        {'length': 4100, 'width': 1720, 'height': 1150, 'weight': 1865 / 1000},
        {'length': 3870, 'width': 2890, 'height': 1020, 'weight': 4085 / 1000},
    ]
    weightSum = sum(map(lambda x: x['weight'], cargo))
    
    
    for index, item in enumerate(cargo):
        if index:
            prev = cargo[index - 1]
            item['ct'] = (item['length'] + prev['length']) / 2 + prev['ct'] + delta[index]
        else:
            item['ct'] = item['length'] / 2 + delta[index]
    
    
    # Продольное смещение грузов в вагоне - в мм.
    Lc = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) / weightSum)
    
    # Продольное смещение грузов с вагоном - в мм.
    Lcv = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) + Qv * L/2) / (weightSum + Qv)